# HLMA 408: Lois gaussiennes et visualisation interactive

***
> __Auteur__: Joseph Salmon <joseph.salmon@umontpellier.fr>

Inspiré des codes :
- https://medium.com/kapernikov/ipywidgets-with-matplotlib-93646718eb84
- https://medium.com/@jdchipox/how-to-interact-with-jupyter-33a98686f24e

In [1]:
# pip install download # si besoin, décommenter et exécuter cette cellule.

In [2]:
# pip install 'ipywidgets==7.5'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, laplace, t, f, expon, beta, cauchy
import ipywidgets
import seaborn as sns

In [4]:
import ipywidgets
ipywidgets.__version__

'7.5.1'

In [5]:
%matplotlib widget

In [6]:
distributions_dict = {'Gaussienne': norm,
                      'Laplace': laplace,
                      'Exponentielle': expon,
                      'Cauchy': cauchy,
                      }
# plus d'exemples ici: https://docs.scipy.org/doc/scipy/reference/stats.html


## Visualisation : densités et tirages aléatoires

In [7]:
def make_box_layout():
    return ipywidgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
    )

In [8]:
class Sines(ipywidgets.HBox):

    def __init__(self):
        super().__init__()
        output = ipywidgets.Output()
        self.n_samples = 30
        self.ranges = (-5, 5)  # Borne d'observation
        
        self.x = np.linspace(self.ranges[0], self.ranges[1], num=400)
        self.mu, self.sigma = 0, 1
        self.distribution = distributions_dict['Gaussienne']
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.size = 3
        self.initial_color = '#3646E3'
        self.jitter = 0.04
        self.params = dict(jitter=self.jitter, size=self.size,
                           color=self.initial_color, alpha=0.70)

        with output:
            self.fig, self.ax = plt.subplots(2, 1, sharex=True,
                                             constrained_layout=True, figsize=(4, 4))
        self.line, = self.ax[0].plot(self.x,
                                     self.distribution.pdf(
                                         self.x, loc=self.mu, scale=self.sigma),
                                     self.initial_color)
        
        
        self.fig.canvas.set_window_title('My title')
        self.ax[0].set_xlabel('Densité')
        self.ax[0].set_xlim(self.ranges[0], self.ranges[1])
        self.ax[0].set_ylim(0, 0.6)
        
        self.ax[1].set_xlabel('Tirage')
        self.ax[1].set_xlim(self.ranges[0], self.ranges[1])
        self.ax[1].axes.get_yaxis().set_visible(False)

        self.points = sns.stripplot(
            x=self.samples, ax=self.ax[1], **self.params)
        self.fig.canvas.toolbar_position = 'bottom'

        # define widgets

        n_samples_slider = ipywidgets.IntSlider(
            value=self.n_samples, min=1, max=500, step=1, description="Nb tirages")
        mu_slider = ipywidgets.FloatSlider(
            value=0, min=-2, max=2, step=0.1, description='$\mu$')
        sigma_slider = ipywidgets.FloatSlider(
            value=1, min=1.1, max=5, step=0.1, description='$\sigma$')
        color_picker = ipywidgets.ColorPicker(
            value=self.initial_color,
            description='Couleur')
        jitter_slider = ipywidgets.FloatSlider(
            value=self.jitter, min=0.005, max=0.3, step=0.005, description='jitter')
        text_distribution = ipywidgets.Dropdown(
            options=list(distributions_dict),
            value='Gaussienne',
            description='Distribution')
        int_range_slider = ipywidgets.FloatRangeSlider(
            value=(-5, 5),
            min=-5, max=5, step=0.1,
            description="Amplitude")

        controls = ipywidgets.VBox([
            n_samples_slider,
            mu_slider,
            sigma_slider,
            text_distribution,
            color_picker,
            int_range_slider,
            jitter_slider
        ])

        controls.layout = make_box_layout()

        out_box = ipywidgets.Box([output])
        output.layout = make_box_layout()

        # Observe stuff
        n_samples_slider.observe(self.update_n_samples, 'value')
        mu_slider.observe(self.update_mu, 'value')
        sigma_slider.observe(self.update_sigma, 'value')
        color_picker.observe(self.line_color, 'value')
        text_distribution.observe(self.update_text_distribution, 'value')
        int_range_slider.observe(self.update_range_slider, 'value')
        jitter_slider.observe(self.update_jitter_slider, 'value')

        # Add to children
        self.children = [controls, output]

    def update_mu(self, change):
        """Draw line in plot."""
        self.mu = change.new
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.line.set_ydata(self.distribution.pdf(self.x, loc=self.mu, scale=self.sigma))
        self.ax[1].clear()  # <-- clear axes
        self.points = sns.stripplot(
            x=self.samples, ax=self.ax[1], **self.params)
        self.ax[1].set_xlabel('Tirage')
        self.fig.canvas.draw()

    def update_sigma(self, change):
        """Draw line in plot."""
        self.sigma = change.new
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.line.set_ydata(self.distribution.pdf(
            self.x, loc=self.mu, scale=self.sigma))
        self.ax[1].clear()  # <-- clear axes
        self.points = sns.stripplot(
            x=self.samples, ax=self.ax[1], size=self.size, color=self.initial_color)
        self.ax[1].set_xlim(self.ranges)
        self.ax[1].set_xlabel('Tirage')
#         self.fig.canvas.draw()

    def update_n_samples(self, change):
        """Draw."""
        self.n_samples = change.new
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.ax[1].clear()  # <-- clear axes
        np.random.seed(123)
        self.points = sns.stripplot(
            x=self.samples, ax=self.ax[1], **self.params)
        self.ax[1].set_xlim(self.ranges)
        self.ax[1].set_xlabel('Tirage')
#         self.fig.canvas.draw()

    def update_jitter_slider(self, change):
        self.params['jitter'] = change.new
        self.ax[1].clear()  # <-- clear axes
        self.points = sns.stripplot(
            x=self.samples, ax=self.ax[1], **self.params)
        self.ax[1].set_xlabel('Tirage')

    def line_color(self, change):
        self.initial_color = change.new
        self.line.set_color(self.initial_color)
        self.ax[1].set_color(self.initial_color)

    def update_range_slider(self, change):
        self.ranges = change.new
        self.line.set_ydata(self.distribution.pdf(
            self.x, loc=self.mu, scale=self.sigma))
        self.ax[0].set_xlim(self.ranges)
        self.ax[1].clear()  # <-- clear axes
        np.random.seed(123)
        self.points = sns.stripplot(
            x=self.samples, ax=self.ax[1], size=self.size, color=self.initial_color)
        self.ax[1].set_xlabel('Tirage')

    def update_text_distribution(self, change):
        self.distribution = distributions_dict[change.new]
        self.line.set_ydata(self.distribution.pdf(
            self.x, loc=self.mu, scale=self.sigma))

        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.ax[0].set_xlim(self.ranges)
        self.ax[1].clear()  # <-- clear axes
        np.random.seed(123)
        self.points = sns.stripplot(
            x=self.samples, ax=self.ax[1], size=self.size, color=self.initial_color)
        self.ax[1].set_xlim(self.ranges)
        self.ax[1].set_xlabel('Tirage')
        self.fig.canvas.draw()


Sines()

Sines(children=(VBox(children=(IntSlider(value=30, description='Nb tirages', max=500, min=1), FloatSlider(valu…